In [1]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include(joinpath("/home/jzh/Dropbox/Research/",
"Data-driven_estimation_inverse_optimization", 
"Experiments/InverseVIsTraffic/Julia_files/load_network_uni-class.jl"))

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("East_Massachusetts_Oct_MD")

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

In [2]:
function MSA(coeffs) 
    polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

    function BPR(x)
        bpr = similar(x)
        for i=1:length(bpr)
            bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
        end
        return bpr
    end

    function all_or_nothing(travel_time)
        state = []
        path = []
        x = zeros(size(start_node))

        for r=1:size(travel_demand)[1]
            # for each origin node r, find shortest paths to all destination nodes
            state = dijkstra_shortest_paths(graph, travel_time, r)

            for s=1:size(travel_demand)[2]
                # for each destination node s, find the shortest-path vector
                # load travel demand
                x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
            end
        end

        return x
    end

    # Finding a starting feasible solution
    travel_time = BPR(zeros(number_of_links))
    xl = all_or_nothing(travel_time)

    max_iter_no = 1e3
    l = 1
    #average_excess_cost = 1
    tol = 1e-5

    while l < max_iter_no
        l += 1

        xl_old = xl

        # Finding yl
        travel_time = BPR(xl)

        yl = all_or_nothing(travel_time)

        xl = xl + (yl - xl)/l

        xl_new = xl

        relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

        if relative_gap < tol
            break
        end
    end
    
    return xl
end

MSA (generic function with 1 method)

In [3]:
# getting the coefficients of the costs
coeffs_dict_Oct_MD = readall("../temp_files/coeffs_dict_Oct_MD.json")
coeffs_dict_Oct_MD = JSON.parse(coeffs_dict_Oct_MD)

xl = Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}}()

deg_grid = [4:8]
c_grid = .5:.5:3.
lamb_grid = [10. .^(-3:4)]

for deg in deg_grid
    for c in c_grid
        for lam in lamb_grid
            coeffs_1 = coeffs_dict_Oct_MD["($(deg),$(c),$(lam),1)"]
            coeffs_2 = coeffs_dict_Oct_MD["($(deg),$(c),$(lam),2)"]
            coeffs_3 = coeffs_dict_Oct_MD["($(deg),$(c),$(lam),3)"]
            xl[(deg, c, lam, 1)] = MSA(coeffs_1)
            xl[(deg, c, lam, 2)] = MSA(coeffs_2)
            xl[(deg, c, lam, 3)] = MSA(coeffs_3)
        end
    end
end

In [4]:
lamb_grid

8-element Array{Float64,1}:
     0.001
     0.01 
     0.1  
     1.0  
    10.0  
   100.0  
  1000.0  
 10000.0  

In [5]:
outfile = open("../temp_files/uni-class_traffic_assignment_MSA_flows_Oct_MD.json", "w")

JSON.print(outfile, xl)

close(outfile)

In [6]:
xl

Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}} with 720 entries:
  (8,2.5,10.0,3)    => [2240.9002486249997,2113.118224149997,2654.5873753050023…
  (4,1.0,0.01,2)    => [2250.685836652998,2107.4604444019974,2644.8017862950005…
  (4,3.0,0.001,1)   => [19257.815162407034,137.43006782499995,171.2816571099996…
  (8,2.0,1000.0,3)  => [2240.9002486249997,2112.7849578159994,2654.587375305002…
  (6,1.5,0.1,1)     => [2245.793043130001,2111.0798733279994,2649.694580799999,…
  (6,1.5,100.0,1)   => [2240.9002486249988,2111.537146046001,2654.5873753050005…
  (4,2.0,1000.0,3)  => [2260.471061310003,2102.930954629997,2632.420428359996,2…
  (5,2.5,10.0,1)    => [2255.578266804998,2103.950130041004,2637.313222864997,2…
  (7,0.5,1.0,3)     => [2231.114659614996,2115.5092175310037,2664.372964315,273…
  (5,0.5,100.0,2)   => [2236.0074541200033,2114.594672279001,2659.48016981,2735…
  (7,1.5,1.0,3)     => [2240.900248624999,2112.7849578159994,2654.5873753050023…
  (4,0.5,0.01,1)    => [2240.900248624

In [7]:
xl[(8,2.5,0.1,2)]

24-element Array{Float64,1}:
 20984.8     
    52.0526  
    60.5489  
 20948.4     
     0.0     
     0.0     
 20631.5     
    72.3888  
  2822.1     
  4998.61    
     0.0     
 18039.4     
    79.665   
     2.52439 
     7.07523 
     0.000983
 20393.7     
    30.5813  
    13.6518  
  1560.48    
    12.127   
 19281.4     
    21.243   
 20339.0     